## 구멍난 상추 사진

In [5]:
import cv2
import numpy as np
import random
import glob
import os

# 이미지 폴더 경로
input_folder = r'C:\hole\img'
output_folder = r'C:\hole\img_result'

# 구멍을 채울 이미지 (fill_image)
fill_image_path = 'fill.jpg'
fill_image = cv2.imread(fill_image_path)

# 이미지가 제대로 불러왔는지 확인
if fill_image is None:
    raise FileNotFoundError("채울 이미지를 불러올 수 없습니다. 경로를 확인하세요.")

# 폴더 내의 모든 이미지 파일 불러오기
image_paths = glob.glob(os.path.join(input_folder, '*.jpg'))

# 이미지 처리
for image_path in image_paths:
    # 이미지 불러오기
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 불러올 수 없습니다: {image_path}")
        continue

    # BGR 이미지를 HSV로 변환
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 초록색 범위 설정 (HSV)
    lower_green = np.array([20, 40, 40])  # 초록색 하한 값
    upper_green = np.array([90, 255, 255])  # 초록색 상한 값

    # 초록색 영역 마스크 생성
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)

    # 초록색 영역의 좌표들 찾기
    green_coords = np.column_stack(np.where(green_mask > 0))

    # 이미지 하나에 대해 5개의 변형된 이미지를 생성
    for variation in range(15):
        # 구멍 개수 설정
        num_holes = 3  # 원하는 구멍 개수
        created_holes = 0  # 생성된 구멍 개수

        # 구멍이 생길 수 있는 영역을 제한 (가장자리 여백 추가)
        border_margin = 10  # 구멍이 가장자리에서 떨어져야 할 최소 거리

        # 구멍 생성
        result = image.copy()  # 결과 이미지 복사본 생성
        for _ in range(num_holes):
            if len(green_coords) == 0:
                print("초록색 영역이 없습니다.")
                break

            while True:
                # 초록색 영역에서 랜덤한 위치 선택
                coord_idx = random.randint(0, len(green_coords) - 1)
                center_y, center_x = green_coords[coord_idx]

                # 구멍이 가장자리에서 떨어진 위치에 있는지 확인
                if center_x > border_margin and center_x < image.shape[1] - border_margin and center_y > border_margin and center_y < image.shape[0] - border_margin:
                    break  # 가장자리에서 충분히 떨어진 경우에만 구멍을 생성

            # 불규칙한 타원형 다각형 모양의 구멍을 생성
            num_vertices = random.randint(10, 20)  # 타원형 다각형 꼭짓점 개수
            axis_x = random.randint(20, 40)  # 타원의 가로축 반지름
            axis_y = random.randint(10, 50)  # 타원의 세로축 반지름
            angle_offset = random.uniform(0, 360)  # 타원의 회전 각도

            polygon_points = []
            valid_polygon = True  # 구멍이 초록색 영역 내에 있는지 확인하는 플래그

            for i in range(num_vertices):
                angle = 2 * np.pi * i / num_vertices + angle_offset  # 각도를 고르게 분포시키며 회전 추가
                r_x = axis_x + random.randint(-10, 20)  # 가로축 반경에 약간의 랜덤 변동 추가
                r_y = axis_y + random.randint(-10, 20)  # 세로축 반경에 약간의 랜덤 변동 추가
                x = int(center_x + r_x * np.cos(angle))
                y = int(center_y + r_y * np.sin(angle))

                # 구멍의 꼭짓점이 초록색 영역 내에 있는지 확인
                if green_mask[y, x] == 0:
                    valid_polygon = False  # 구멍이 초록색을 벗어나면 false로 설정
                    break

                polygon_points.append([x, y])

            if not valid_polygon:
                # 구멍이 초록색 영역을 벗어나면 다시 구멍을 생성
                continue

            polygon_points = np.array(polygon_points, np.int32).reshape((-1, 1, 2))

            # 다각형 마스크 생성
            mask = np.zeros_like(image)
            cv2.fillPoly(mask, [polygon_points], (255, 255, 255))

            # 다각형 영역 구하기
            x_min = np.min(polygon_points[:, 0, 0])
            x_max = np.max(polygon_points[:, 0, 0])
            y_min = np.min(polygon_points[:, 0, 1])
            y_max = np.max(polygon_points[:, 0, 1])

            # 채울 이미지를 다각형 크기에 맞게 자름
            fill_image_resized = cv2.resize(fill_image, (x_max - x_min, y_max - y_min))

            # 마스크로 다각형 이미지 적용
            fill_mask = np.zeros_like(fill_image_resized)
            cv2.fillPoly(fill_mask, [np.array([[(x - x_min, y - y_min) for x, y in polygon_points[:, 0]]], np.int32)], (255, 255, 255))

            # 다각형으로 잘린 이미지를 마스크로 적용
            fill_circle = cv2.bitwise_and(fill_image_resized, fill_mask)

            # 구멍 크기에 맞춰서 다각형 이미지를 채움
            region = result[y_min:y_max, x_min:x_max]
            final_result = np.where(mask[y_min:y_max, x_min:x_max] > 0, fill_circle, region)

            result[y_min:y_max, x_min:x_max] = final_result

            created_holes += 1  # 구멍 생성 개수 증가

        # 구멍이 생성된 경우에만 결과 이미지 저장
        if created_holes > 0:
            # 결과 이미지를 RGB로 변환하여 저장
            result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

            # 출력 경로 지정 및 저장 (각 변형에 번호 추가)
            output_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_variation_{variation+1}.jpg"
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, cv2.cvtColor(result_rgb, cv2.COLOR_RGB2BGR))
            print(f"이미지를 '{output_path}'로 저장했습니다.")
        else:
            print(f"구멍이 생성되지 않아 이미지를 저장하지 않습니다: {image_path}")

구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7642.JPG
이미지를 'C:\hole\img_result\IMG_7642_variation_2.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7642.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7642.JPG
이미지를 'C:\hole\img_result\IMG_7642_variation_5.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7642.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7642.JPG
이미지를 'C:\hole\img_result\IMG_7642_variation_8.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7642_variation_9.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7642.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7642.JPG
이미지를 'C:\hole\img_result\IMG_7642_variation_12.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7642_variation_13.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7642.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7642.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7643.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7643.JPG
이미지를 'C:\hole\img_result\IMG_7643_variation_3.jpg'로 저장했

이미지를 'C:\hole\img_result\IMG_7651_variation_10.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7651_variation_11.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7651_variation_12.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7651_variation_13.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7651_variation_14.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7651.JPG
이미지를 'C:\hole\img_result\IMG_7652_variation_1.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7652.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7652.JPG
이미지를 'C:\hole\img_result\IMG_7652_variation_4.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7652_variation_5.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7652_variation_6.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7652_variation_7.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7652_variation_8.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7652.JPG
이미지를 'C:\hole\img_result\IMG_7652_variation_10.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7652_variation_11.jpg'로 저장했습니다.
구

이미지를 'C:\hole\img_result\IMG_7661_variation_4.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7661.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7661.JPG
이미지를 'C:\hole\img_result\IMG_7661_variation_7.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7661_variation_8.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7661_variation_9.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7661_variation_10.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7661.JPG
이미지를 'C:\hole\img_result\IMG_7661_variation_12.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7661_variation_13.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7661_variation_14.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7661_variation_15.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7662.JPG
이미지를 'C:\hole\img_result\IMG_7662_variation_2.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7662_variation_3.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7662_variation_4.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7662_variation_5.jpg'로 저장했습니다.
구멍이

이미지를 'C:\hole\img_result\IMG_7670_variation_12.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7670_variation_13.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7670_variation_14.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7670_variation_15.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7671_variation_1.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7671.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7671.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7671.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7671.JPG
이미지를 'C:\hole\img_result\IMG_7671_variation_6.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7671_variation_7.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7671_variation_8.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7671_variation_9.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7671.JPG
이미지를 'C:\hole\img_result\IMG_7671_variation_11.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7671_variation_12.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7671_variation_13.jpg'로 저장했습니다.
이미지를 'C:

이미지를 'C:\hole\img_result\IMG_7680_variation_5.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7680_variation_6.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7680_variation_7.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7680_variation_8.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7680_variation_9.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7680_variation_10.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7680.JPG
이미지를 'C:\hole\img_result\IMG_7680_variation_12.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7680_variation_13.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7680_variation_14.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7680_variation_15.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7681.JPG
이미지를 'C:\hole\img_result\IMG_7681_variation_2.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7681_variation_3.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7681_variation_4.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7681_variation_5.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7681_variation_6.jpg

...

구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7744.JPG
이미지를 'C:\hole\img_result\IMG_7744_variation_2.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7744_variation_3.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7744_variation_4.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7744.JPG
이미지를 'C:\hole\img_result\IMG_7744_variation_6.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7744_variation_7.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7744.JPG
이미지를 'C:\hole\img_result\IMG_7744_variation_9.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7744_variation_10.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7744.JPG
이미지를 'C:\hole\img_result\IMG_7744_variation_12.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7744.JPG
이미지를 'C:\hole\img_result\IMG_7744_variation_14.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7744_variation_15.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7745_variation_1.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7745.JPG
이미지를 'C:\hole\img_

이미지를 'C:\hole\img_result\IMG_7753_variation_11.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7753_variation_12.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7753_variation_13.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7753_variation_14.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7753.JPG
이미지를 'C:\hole\img_result\IMG_7754_variation_1.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7754_variation_2.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7754.JPG
이미지를 'C:\hole\img_result\IMG_7754_variation_4.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7754_variation_5.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7754_variation_6.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7754.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7754.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7754.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7754.JPG
이미지를 'C:\hole\img_result\IMG_7754_variation_11.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7754.JPG
이미지를 'C:\hole\img_result

이미지를 'C:\hole\img_result\IMG_7763_variation_5.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7763_variation_6.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7763_variation_7.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7763_variation_8.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7763.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7763.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7763.JPG
이미지를 'C:\hole\img_result\IMG_7763_variation_12.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7763_variation_13.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7763_variation_14.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7763.JPG
이미지를 'C:\hole\img_result\IMG_7764_variation_1.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7764.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7764.JPG
이미지를 'C:\hole\img_result\IMG_7764_variation_4.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7764.JPG
이미지를 'C:\hole\img_result\IMG_7764_variation_6.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\I

이미지를 'C:\hole\img_result\IMG_7773_variation_1.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7773.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7773.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7773.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7773.JPG
이미지를 'C:\hole\img_result\IMG_7773_variation_6.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7773_variation_7.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7773.JPG
이미지를 'C:\hole\img_result\IMG_7773_variation_9.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7773_variation_10.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7773.JPG
이미지를 'C:\hole\img_result\IMG_7773_variation_12.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7773.JPG
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7773.JPG
이미지를 'C:\hole\img_result\IMG_7773_variation_15.jpg'로 저장했습니다.
구멍이 생성되지 않아 이미지를 저장하지 않습니다: C:\hole\img\IMG_7774.JPG
이미지를 'C:\hole\img_result\IMG_7774_variation_2.jpg'로 저장했습니다.
이미지를 'C:\hole\img_result\IMG_7774_variat

IndexError: index 3036 is out of bounds for axis 0 with size 3024